<a href="https://colab.research.google.com/github/Ripa-Shah/Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price/blob/main/MIS584_Cryptocurrency_sentimentdata_and_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install pyspark praw nltk pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.6 MB/s eta 0:00:00


In [17]:
%pwd

'/content'

In [18]:
pip install tweepy praw requests pandas textblob vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [20]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [22]:
# List of time filters to check, ordered from newest to oldest
TIME_FILTERS = ["day", "week", "month", "year"]

# Define the start date (as before)
start_date_str = "2025-08-15 00:00:00"
start_timestamp = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()

# Initialize a set to prevent processing the same post multiple times
processed_ids = set()

In [106]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize Reddit API
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Define cryptocurrency keywords to track
crypto_keywords = {
    "bitcoin": "BTC",
    "btc": "BTC",
    "Bitcoin": "BTC",
    "ethereum": "ETH",
    "eth": "ETH",
    "Ethereum": "ETH",
    "cardano": "ADA",
    "ada": "ADA",
    "Cardano": "ADA",
    "dogecoin": "DOGE",
    "doge": "DOGE",
    "Dogecoin": "DOGE",
    "solana": "SOL",
    "sol": "SOL",
    "Solana": "SOL"
}

# CSV file path
CSV_FILE = "reddit_crypto_sentiment.csv"

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize CSV file (only once)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)

print("🔄 Starting Reddit post & comment sentiment streaming...")

# ----------------------------------------------------------------------
# NEW FETCHING LOGIC
# ----------------------------------------------------------------------

records = []
print(f"Fetching posts since {start_date_str}...")

# 1. Iterate over the time filters
for time_filter in TIME_FILTERS:
    print(f"\n--- Checking Top posts for: {time_filter.upper()} ---")

    # Fetch the top posts for the current time filter (max 1000 per request)
    # The 'limit=None' tells PRAW to fetch up to its internal limit (usually 1000)
    for post in reddit.subreddit("CryptoCurrency").top(time_filter=time_filter, limit=None):
        post_timestamp = post.created_utc

        # Check 1: Has this post been processed already? (Prevents duplicates from different time filters)
        if post.id in processed_ids:
            continue

        # Check 2: Is the post within the required August 1st to Today range?
        if post_timestamp >= start_timestamp:

            # Mark the post as processed
            processed_ids.add(post.id)

            # --- Start Existing Processing Logic (Post) ---
            title = post.title
            review = post.selftext if post.selftext else ""
            user_id = str(post.author) if post.author else "Anonymous"
            date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")

            # Detect which crypto is mentioned
            text_lower = (title + " " + review).lower()
            found_coin = None
            # ... (Rest of your crypto keyword detection logic)
            for keyword, symbol in crypto_keywords.items():
                if keyword in text_lower:
                    found_coin = symbol
                    break
            if not found_coin:
                continue

            # Sentiment for post
            sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
            records.append([date, user_id, "post", title, found_coin, review, sentiment])
            print(f"✅ Found Post ({time_filter}): {date} | {found_coin} | Sentiment: {sentiment:+.3f}")

            # --- Start Existing Comment Processing Logic ---
            post.comments.replace_more(limit=0)
            for comment in post.comments.list()[:10]:
                # ... (Your existing comment processing and sentiment logic goes here)
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")

                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue

                comment_lower = comment_text.lower()
                comment_coin = found_coin
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break

                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])

        elif time_filter == "year" and post_timestamp < start_timestamp:
            # Only break out of the loop if we're on the longest time filter and found old data.
            # Shorter filters ("day", "week") will naturally stop finding posts as they go too far back.
            print(f"--- Stopping fetch: Hit post older than {start_date_str} on the 'year' filter. ---")
            break

# ----------------------------------------------------------------------
# END NEW FETCHING LOGIC
# ----------------------------------------------------------------------

# --- Final Saving Logic (Run once after all fetching is complete) ---
if records:
    df = pd.DataFrame(records, columns=["date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"])
    df.to_csv(CSV_FILE, mode="a", header=False, index=False)
    print(f"\n✅ FINAL SAVE: Collected and saved {len(records)} unique entries to {CSV_FILE}")

print("Processing complete.")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



🔄 Starting Reddit post & comment sentiment streaming...
Fetching posts since 2025-08-15 00:00:00...

--- Checking Top posts for: DAY ---


/tmp/ipython-input-3489740996.py:77: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



✅ Found Post (day): 2025-11-15 21:53:13 | BTC | Sentiment: +0.846


/tmp/ipython-input-3489740996.py:101: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

✅ Found Post (week): 2025-11-16 01:00:44 | ETH | Sentiment: +0.954
✅ Found Post (week): 2025-11-14 05:49:38 | BTC | Sentiment: -0.402


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



✅ Found Post (week): 2025-11-11 19:46:01 | BTC | Sentiment: +0.956

--- Checking Top posts for: MONTH ---


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




--- Checking Top posts for: YEAR ---
--- Stopping fetch: Hit post older than 2025-08-15 00:00:00 on the 'year' filter. ---

✅ FINAL SAVE: Collected and saved 35 unique entries to reddit_crypto_sentiment.csv
Processing complete.


In [107]:
!pip install pyspark

In [110]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("MIS584_Spark_Basics")
sc = SparkContext(conf = conf)

In [111]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MIS584_Spark_Basics").getOrCreate()

In [112]:

# you can get a spark context from a spark session
sc = spark.sparkContext

In [ ]:
#download the dataset crypto_prices_historical.csv and reddit_crypto_Sentiment.csv

In [113]:
# create a spark dataframe from a pandas dataframe
import pandas as pd

## first, read the external csv file into a pandas dataframe
crypto_prices = pd.read_csv('crypto_prices_historical.csv')

## then, convert the pandas dataframe to a spark dataframe
df = spark.createDataFrame(crypto_prices)
df.show(n=10)

## we can also convert a spark dataframe to a pandas dataframe
pdf = df.toPandas()
print(pdf.head(20))

+---------------+-------+-----------+
|      timestamp| symbol|      price|
+---------------+-------+-----------+
|8/15/2025 17:00|BTC-USD|117678.0078|
|8/15/2025 18:00|BTC-USD|117189.3281|
|8/15/2025 19:00|BTC-USD|117005.4922|
|8/15/2025 20:00|BTC-USD|117355.7578|
|8/15/2025 21:00|BTC-USD|117377.7891|
|8/15/2025 22:00|BTC-USD|117777.8281|
|8/15/2025 23:00|BTC-USD|117416.2891|
| 8/16/2025 0:00|BTC-USD|117834.2734|
| 8/16/2025 1:00|BTC-USD|117899.3203|
| 8/16/2025 2:00|BTC-USD| 117597.875|
+---------------+-------+-----------+
only showing top 10 rows

          timestamp   symbol        price
0   8/15/2025 17:00  BTC-USD  117678.0078
1   8/15/2025 18:00  BTC-USD  117189.3281
2   8/15/2025 19:00  BTC-USD  117005.4922
3   8/15/2025 20:00  BTC-USD  117355.7578
4   8/15/2025 21:00  BTC-USD  117377.7891
5   8/15/2025 22:00  BTC-USD  117777.8281
6   8/15/2025 23:00  BTC-USD  117416.2891
7    8/16/2025 0:00  BTC-USD  117834.2734
8    8/16/2025 1:00  BTC-USD  117899.3203
9    8/16/2025 2:00  B

In [114]:
# create a spark dataframe from a pandas dataframe
import pandas as pd

## first, read the external csv file into a pandas dataframe
df_reddit_crypto = pd.read_csv('reddit_crypto_sentiment.csv')

## then, convert the pandas dataframe to a spark dataframe
df_reddit_crypto = spark.createDataFrame(pdf)
df_reddit_crypto.show(n=5)

## we can also convert a spark dataframe to a pandas dataframe
pdf_reddit = df_reddit_crypto.toPandas()
print(pdf_reddit.head(20))

+---------------+-------+-----------+
|      timestamp| symbol|      price|
+---------------+-------+-----------+
|8/15/2025 17:00|BTC-USD|117678.0078|
|8/15/2025 18:00|BTC-USD|117189.3281|
|8/15/2025 19:00|BTC-USD|117005.4922|
|8/15/2025 20:00|BTC-USD|117355.7578|
|8/15/2025 21:00|BTC-USD|117377.7891|
+---------------+-------+-----------+
only showing top 5 rows

          timestamp   symbol        price
0   8/15/2025 17:00  BTC-USD  117678.0078
1   8/15/2025 18:00  BTC-USD  117189.3281
2   8/15/2025 19:00  BTC-USD  117005.4922
3   8/15/2025 20:00  BTC-USD  117355.7578
4   8/15/2025 21:00  BTC-USD  117377.7891
5   8/15/2025 22:00  BTC-USD  117777.8281
6   8/15/2025 23:00  BTC-USD  117416.2891
7    8/16/2025 0:00  BTC-USD  117834.2734
8    8/16/2025 1:00  BTC-USD  117899.3203
9    8/16/2025 2:00  BTC-USD  117597.8750
10   8/16/2025 3:00  BTC-USD  117511.2188
11   8/16/2025 4:00  BTC-USD  117906.6094
12   8/16/2025 5:00  BTC-USD  117679.5547
13   8/16/2025 6:00  BTC-USD  117480.6094
14 

In [115]:
from pyspark.sql.types import *

# create a spark dataframe from a csv file with inferred schema:
df_reddit_crypto = spark.read.csv('reddit_crypto_sentiment.csv', header=True, inferSchema=True) # Spark will automatically detect the data type of each column.
df_reddit_crypto.show()

# print the schema (column names and types of a dataframe)
print('Inferred Schema of the DataFrame:')
df_reddit_crypto.printSchema()

# create a spark dataframe from a csv file with specified schema:
schema = StructType([
    StructField('date', TimestampType(), True),
    StructField('user_id', StringType(), True),
    StructField('tyoe', StringType(), True),
    StructField('title', StringType(), True),
    StructField('cryptocurrency', StringType(), True),
    StructField('review',StringType(),True),
    StructField('sentiment_score',DoubleType(),True)
])
df_reddit_crypto = spark.read.csv('reddit_crypto_sentiment.csv', header=True, schema=schema)
print('Specified Schema of the DataFrame:')
df_reddit_crypto.printSchema()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|                date|             user_id|                type|               title|      cryptocurrency|              review|sentiment_score|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
| 2025-11-15 21:53:13|          ReignNFire|                post|ZEC and the weird...|                 BTC|Throw out privacy...|           NULL|
|What ZCash has go...| so what do we do...|     not widely used| but based off Bi...| my tin foil hat ...|                NULL|           NULL|
|What do you think...| I find it hard t...| let me get some ...|              0.8458|                NULL|                NULL|           NULL|
| 2025-11-15 22:18:03|      Top-Squash6558|             comment|ZEC and the weird...|                 BTC|Just seems like a...|         

In [116]:
# get the number of rows for a dataframe
print()
print('Number of rows of the DataFrame:', df_reddit_crypto.count())

# get the number of columns and column ames for a dataframe
print()
print('Number of columns of the DataFrame:', len(df_reddit_crypto.columns))
print('Column names of the DataFrame:', df_reddit_crypto.columns)


Number of rows of the DataFrame: 93

Number of columns of the DataFrame: 7
Column names of the DataFrame: ['date', 'user_id', 'tyoe', 'title', 'cryptocurrency', 'review', 'sentiment_score']


In [117]:
# drop duplicates from a dataframe
print('Number of rows before dropping duplicates:', df_reddit_crypto.count())
df_reddit_crypto.dropDuplicates().count()
print('Number of rows after dropping duplicates:', df_reddit_crypto.dropDuplicates().count())

Number of rows before dropping duplicates: 93
Number of rows after dropping duplicates: 62


In [118]:
#print statistics of data frame
df_reddit_crypto.describe()

DataFrame[summary: string, user_id: string, tyoe: string, title: string, cryptocurrency: string, review: string, sentiment_score: string]

In [119]:
#rename column tyoe to type

df_reddit_crypto = df_reddit_crypto.withColumnRenamed("tyoe", "type")

# Display the new schema to confirm the change
df_reddit_crypto.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- cryptocurrency: string (nullable = true)
 |-- review: string (nullable = true)
 |-- sentiment_score: double (nullable = true)



In [120]:
from pyspark.sql.functions import current_timestamp, to_timestamp


In [121]:
#fill missing values with keyword missing and for sentiment_score ->0.0
df_reddit_crypto_filled = df_reddit_crypto.na.fill(value=0.0, subset=['sentiment_score'])
#df_reddit_crypto.na.fill(current_timestamp(), subset=["date"])
df_reddit_crypto_missing = df_reddit_crypto_filled.na.fill(value='missing', subset=['cryptocurrency','title','review','user_id','type'])
#df_reddit_crypto.na.fill(value='1/1/1930', subset=['date']).show(100)

In [127]:
# List of values to EXCLUDE
excluded_cryptos = ["BTC", "ETH", "SOL", "ADA","DOGE","missing"]  # Adjusted BTH to BTC for clarity

# Find rows where the cryptocurrency IS NOT in the excluded list
df_other_cryptos = df_reddit_crypto_missing.filter(~col("cryptocurrency").isin(excluded_cryptos)
)

df_reddit_main_cryptos = df_reddit_crypto_missing.filter(col("cryptocurrency").isin(excluded_cryptos))
print("--Main cryptos")
df_reddit_main_cryptos.show()
df_reddit_main_cryptos_count = df_reddit_main_cryptos.count()
print("--Main cryptos to count:",{df_reddit_main_cryptos_count})


# Display the result
print("--- Rows where cryptocurrency IS NOT BTC, ETH, SOL, or ADA ---")
df_other_cryptos.show()

# count the result
df_other_cryptos_count = df_other_cryptos.count()
print("Total excluded count",{df_other_cryptos_count})

# Show the distinct names of the excluded cryptocurrencies
#df_to_save = df_other_cryptos.select("cryptocurrency").distinct()

# Assuming 'df_to_save' is the DataFrame you want to export (e.g., df_other_cryptos)

#CSV_FILE_EXCLUDE= 'exclude_cryptocurrency.csv'
#df_to_save.write.csv(
 #   CSV_FILE_EXCLUDE,
 #   mode="overwrite",      # Options: 'overwrite', 'append', 'ignore', 'error' (default)
 #   header=True            # Writes the column names as the first line of the file
#)


--Main cryptos
+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+---------------+
|               date|             user_id|                type|               title|cryptocurrency|              review|sentiment_score|
+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+---------------+
|2025-11-15 21:53:13|          ReignNFire|                post|ZEC and the weird...|           BTC|Throw out privacy...|            0.0|
|               NULL| I find it hard t...| let me get some ...|              0.8458|       missing|             missing|            0.0|
|2025-11-15 22:18:03|      Top-Squash6558|             comment|ZEC and the weird...|           BTC|Just seems like a...|            0.0|
|               NULL|             missing|             missing|             missing|       missing|             missing|            0.0|
|               NULL| xlm 

In [128]:

# apply the filter transformation on a dataframe
print('Reddit posts and comments positive sentiment score:')
df_reddit_main_cryptos.filter(df_reddit_crypto['sentiment_score'] > 0.0)
df_reddit_main_cryptos.show(n=5)
print('Reddit posts and comments negative sentiment score:')
df_reddit_main_cryptos.filter(df_reddit_crypto['sentiment_score'] < 0.0)
df_reddit_main_cryptos.show(n=5)
print('Reddit posts and comments neutral sentiment score:')
df_reddit_main_cryptos.filter(df_reddit_crypto['sentiment_score'] == 0.0)
df_reddit_main_cryptos.show(n=5)



Reddit posts and comments positive sentiment score:
+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+---------------+
|               date|             user_id|                type|               title|cryptocurrency|              review|sentiment_score|
+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+---------------+
|2025-11-15 21:53:13|          ReignNFire|                post|ZEC and the weird...|           BTC|Throw out privacy...|            0.0|
|               NULL| I find it hard t...| let me get some ...|              0.8458|       missing|             missing|            0.0|
|2025-11-15 22:18:03|      Top-Squash6558|             comment|ZEC and the weird...|           BTC|Just seems like a...|            0.0|
|               NULL|             missing|             missing|             missing|       missing|             missing|      

In [129]:
#Add additional column called sentiment_label
from pyspark.sql.functions import col, when
POSITIVE_THRESHOLD = 0.1
NEGATIVE_THRESHOLD = -0.1

df_reddit_main_cryptos = df_reddit_main_cryptos.withColumn(
    "sentiment_label",
    when(col("sentiment_score") > POSITIVE_THRESHOLD, "Positive")
    .when(col("sentiment_score") < NEGATIVE_THRESHOLD, "Negative")
    .otherwise("Neutral")
)
df_reddit_main_cryptos.show(n=10)


+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+---------------+---------------+
|               date|             user_id|                type|               title|cryptocurrency|              review|sentiment_score|sentiment_label|
+-------------------+--------------------+--------------------+--------------------+--------------+--------------------+---------------+---------------+
|2025-11-15 21:53:13|          ReignNFire|                post|ZEC and the weird...|           BTC|Throw out privacy...|            0.0|        Neutral|
|               NULL| I find it hard t...| let me get some ...|              0.8458|       missing|             missing|            0.0|        Neutral|
|2025-11-15 22:18:03|      Top-Squash6558|             comment|ZEC and the weird...|           BTC|Just seems like a...|            0.0|        Neutral|
|               NULL|             missing|             missing|             missin

In [130]:
# group a dataframe by certain columns and calculate group statistics
from pyspark.sql.functions import max, min, sum, avg, stddev, round, count

print('Top total number of posts and comments posted by a userid:')
df_reddit_count = df_reddit_main_cryptos.groupby('user_id').agg(
    count('review').alias('total_reviews'),
    count('title').alias('total_titles')
)
df_reddit_count_sort = df_reddit_count.sort(col('total_reviews').desc(), col('total_titles').desc())
df_reddit_count_sort.show(n=10)

Top total number of posts and comments posted by a userid:
+--------------------+-------------+------------+
|             user_id|total_reviews|total_titles|
+--------------------+-------------+------------+
|             missing|           32|          32|
|       AutoModerator|            3|           3|
|        not420guilty|            2|           2|
| and this action ...|            2|           2|
|             jwz9904|            2|           2|
|           SirCatDad|            1|           1|
|   Strange-Term-4168|            1|           1|
| how does that no...|            1|           1|
|              0.8548|            1|           1|
|    DepartedQuantity|            1|           1|
+--------------------+-------------+------------+
only showing top 10 rows



In [131]:
# Total number of unique user who have posted comments and post for cryptocurrency
from pyspark.sql.functions import col, countDistinct
df_reddit_crypto_count =  df_reddit_main_cryptos.groupBy("cryptocurrency", "type").agg(
    countDistinct("user_id").alias("unique_user_count")
)

df_reddit_crypto_count.select(['unique_user_count', 'cryptocurrency', 'type'])\
.sort('unique_user_count', 'cryptocurrency', ascending=[False, False]).show(n=15)

+-----------------+--------------+--------------------+
|unique_user_count|cryptocurrency|                type|
+-----------------+--------------+--------------------+
|               22|           BTC|             comment|
|               18|       missing|             missing|
|                7|           ETH|             comment|
|                4|           BTC|                post|
|                1|       missing| and their alt ba...|
|                1|       missing| the exchange wil...|
|                1|       missing|              0.8316|
|                1|       missing|              0.4374|
|                1|       missing| and could be an ...|
|                1|       missing| let me get some ...|
|                1|       missing|               0.913|
|                1|           ETH|                post|
+-----------------+--------------+--------------------+



In [ ]:
#create a bar chart for cryptocurrency

In [ ]:
#create a column chart for number of title and revviews for cryptocurrency

In [ ]:
#create a bar chart to count sentiment score for cryptocurrency

In [ ]:
#https://www.kaggle.com/code/suzanaiacob/sentiment-analysis-of-the-yelp-reviews-data
#https://www.kaggle.com/code/serkanp/imdb-sentiment-analysis-data-exploration
#https://www.kaggle.com/code/ryanholbrook/exercise-binary-classification
https://www.kaggle.com/code?searchQuery=sentiment+data+
https://www.kaggle.com/code/benhamner/exploring-airline-twitter-sentiment-data
https://www.kaggle.com/code/seunowo/sentiment-analysis-twitter-dataset

In [132]:
sc.stop()